In [ ]:
import pandas as pd
import os
from matplotlib import pyplot as plt

DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/HW 5/data'

# Convert Excel to CSV

In [ ]:
# Convert xlsx files to csv files for 2017-2019 (makes it faster to load in the future)
# DONT RUN if csv files already exist

for year in range(2017, 2020):
    print(f"Converting {year}.xlsx to {year}.csv...")
    xlsx_path = os.path.join(DATA_DIR, f"{year}.xlsx")
    csv_path = os.path.join(DATA_DIR, f"{year}.csv")
    data = pd.read_excel(xlsx_path, header=1)
    data.to_csv(csv_path, index=False, encoding='euc-kr')

Converting 2017.xlsx to 2017.csv...
Converting 2018.xlsx to 2018.csv...
Converting 2019.xlsx to 2019.csv...


# Process and Load Data

In [ ]:
def clean_data(df: pd.DataFrame, includes_line: bool = False, includes_total: bool = False, includes_24: bool = True) -> pd.DataFrame:
    # TODO fix verification
    # Verify the structure of the old data
    # expected_columns = ['날짜', '역번호', '역명', '구분', '05~06', '06~07', '07~08',
    #    '08~09', '09~10', '10~11', '11~12', '12~13', '13~14',
    #    '14~15', '15~16', '16~17', '17~18', '18~19', '19~20',
    #    '20~21', '21~22', '22~23', '23~24', '24~']
    # if includes_line:
    #     expected_columns.insert(1, '호선')
    # if includes_total:
    #     expected_columns.append('합계')
    # assert list(df.columns.str.strip()) == expected_columns, "Data structure does not match expected format!"

    # Rename columns to English
    english_columns = ['Date', 'Station Number', 'Station Name', 'Boarding', '05~06', '06~07', '07~08',
    '08~09', '09~10', '10~11', '11~12', '12~13', '13~14',
    '14~15', '15~16', '16~17', '17~18', '18~19', '19~20',
    '20~21', '21~22', '22~23', '23~24']
    if includes_24:
        english_columns.append('24~')
    if includes_line:
        english_columns.insert(1, 'Line')
    if includes_total:
        english_columns.append('Total')
    df.columns = english_columns

    # Remove unnecessary column
    df.drop(columns=['Station Name'], inplace=True)

    # Ensure numeric columns are properly formatted
    col_start = 4 if includes_line else 3
    hours = 20 if includes_24 else 19
    col_end = col_start + hours
    for col in df.columns[col_start:col_end]:
        if df[col].dtype != 'int64':
            df[col] = df[col].astype(str)
            df[col] = pd.to_numeric(df[col].str.strip().str.replace(',', ''), errors='coerce').astype('Int64')
    if not includes_total:
        df["Total"] = df.iloc[:, col_start:col_end].sum(axis=1)

    # Convert Boarding to boolean
    df.Boarding = df.Boarding.apply(lambda x: True if "승차" in x else False)

    # Make sure line column is numeric if exists
    if includes_line and df.Line.dtype != 'int64':
        df.Line = df.Line.apply(lambda x: int(x[0]))

    # Convert Date to datetime
    df.Date = df.Date.str.strip()
    df.Date = pd.to_datetime(df.Date, format='mixed')
    df.set_index('Date', inplace=True)

    return df

In [ ]:
# Test
df = pd.read_csv(os.path.join(DATA_DIR, '2010.csv'), encoding='euc-kr')
clean_data(df)

,Station Number,Boarding,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,...,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,24~,Total
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,150,True,390,400,452,1008,1326,1688,2265,2363,...,2794,2961,2463,2831,2070,2224,1716,804,18,35696
2010-01-01,150,False,187,1169,1171,1715,1984,2396,2209,2149,...,2015,1924,1898,1360,1147,1171,932,548,216,30956
2010-01-01,151,True,114,106,144,160,218,194,272,270,...,822,871,965,853,839,812,594,176,0,9068
2010-01-01,151,False,40,197,229,359,368,328,490,528,...,662,641,643,472,318,250,155,121,13,8134
2010-01-01,152,True,917,714,364,304,306,365,502,754,...,1851,2210,2243,2210,2090,2575,2583,1207,22,24934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31,2825,False,53,93,119,226,181,179,185,239,...,503,568,703,625,454,456,395,268,159,6536
2010-12-31,2826,True,127,186,594,762,421,234,234,330,...,341,393,388,263,205,184,161,67,36,5786
2010-12-31,2826,False,41,60,164,279,202,144,173,214,...,402,523,487,485,365,325,324,243,131,5600


In [ ]:
# Load data between 2008 and 2016
datasets = {}
for year in range(2008, 2017):
    print(f"Processing data for year: {year}...")
    file_path = os.path.join(DATA_DIR, f"{year}.csv")
    data = pd.read_csv(file_path, encoding='euc-kr')
    cleaned_data = clean_data(data)
    datasets[year] = cleaned_data
print("Done!")

Processing data for year: 2008...
Processing data for year: 2009...
Processing data for year: 2010...
Processing data for year: 2011...
Processing data for year: 2012...
Processing data for year: 2013...
Processing data for year: 2014...
Processing data for year: 2015...


/tmp/ipython-input-3088966155.py:6: DtypeWarning: Columns (4,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='euc-kr')


Processing data for year: 2016...
Done!


In [ ]:
# Load 2017
print("Processing data for year: 2017...")
df = pd.read_csv(os.path.join(DATA_DIR, '2017.csv'), encoding='euc-kr')
# Drop unnecessary columns
df.drop(columns=['구분'], inplace=True)

datasets[2017] = clean_data(df, includes_line=True, includes_total=True)

# Load 2018
print("Processing data for year: 2018...")
df = pd.read_csv(os.path.join(DATA_DIR, '2018.csv'), encoding='euc-kr')

datasets[2018] = clean_data(df, includes_line=True, includes_total=True)

# Load 2019
print("Processing data for year: 2019...")
df = pd.read_csv(os.path.join(DATA_DIR, '2019.csv'), encoding='euc-kr')

datasets[2019] = clean_data(df, includes_line=True, includes_total=True)

print("Done!")


Processing data for year: 2017...
Processing data for year: 2018...
Processing data for year: 2019...
Done!


In [ ]:
# Load 2020
print("Processing data for year: 2020...")
df = pd.read_csv(os.path.join(DATA_DIR, '2020.csv'), encoding='euc-kr')

datasets[2020] = clean_data(df, includes_line=True)

# Load 2021
print("Processing data for year: 2021...")
df = pd.read_csv(os.path.join(DATA_DIR, '2021.csv'), encoding='euc-kr')
df.drop(columns=['연번'], inplace=True)

datasets[2021] = clean_data(df, includes_line=True, includes_total=True, includes_24=False)

# Load 2022
print("Processing data for year: 2022...")
df = pd.read_csv(os.path.join(DATA_DIR, '2022.csv'), encoding='euc-kr')
df.drop(columns=['연번'], inplace=True)

datasets[2022] = clean_data(df, includes_line=True, includes_total=False)

# Load 2023
print("Processing data for year 2023...")
df = pd.read_csv(os.path.join(DATA_DIR, '2023.csv'), encoding='euc-kr')
df.drop(columns=['연번'], inplace=True)

datasets[2023] = clean_data(df, includes_line=True)

# Load 2024
print("Processing data for year: 2024...")
df = pd.read_csv(os.path.join(DATA_DIR, '2024.csv'), encoding='euc-kr')
df.drop(columns=['연번'], inplace=True)

datasets[2024] = clean_data(df, includes_line=True)

print("All data processing complete!")

Processing data for year: 2020...
Processing data for year: 2021...
Processing data for year: 2022...


/tmp/ipython-input-2283568355.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_DIR, '2022.csv'), encoding='euc-kr')


Processing data for year 2023...
Processing data for year: 2024...
All data processing complete!


In [ ]:
# Merge all datasets into a single DataFrame
all_data = pd.concat(datasets.values())
all_data

,Station Number,Boarding,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,...,17~18,18~19,19~20,20~21,21~22,22~23,23~24,24~,Total,Line
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,150,True,379,287,371,876,965,1389,1989,2375,...,3495,3055,2952,2726,3307,2584,1059,264,39144,NaN
2008-01-01,150,False,145,707,689,1037,1170,1376,1451,1743,...,2203,2128,1747,1593,1078,744,406,558,27095,NaN
2008-01-01,151,True,131,131,101,152,191,202,275,361,...,1154,1706,1444,1267,928,531,233,974,12722,NaN
2008-01-01,151,False,35,158,203,393,375,460,591,841,...,1303,1190,830,454,284,141,107,185,12124,NaN
2008-01-01,152,True,1287,867,400,330,345,338,595,791,...,2777,2834,2646,2784,2920,2290,802,1559,30358,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,2826,False,14,85,152,477,233,191,191,250,...,465,468,376,261,248,246,175,68,5260,8.0
2024-12-31,2827,True,80,103,332,400,241,241,268,336,...,440,381,247,136,170,140,90,50,5236,8.0
2024-12-31,2827,False,19,109,138,428,219,197,234,266,...,340,374,281,193,212,207,141,128,4589,8.0


# Data Description & Structure

In [17]:
# Key variables and fields
print("\n1. KEY VARIABLES AND FIELDS:")
print("-" * 40)

variables_info = {
    'Date': 'Timestamp for each daily record (datetime index)',
    'Station Number': 'Unique identifier for subway stations (integer)',
    'Boarding': 'Boolean flag: True for boarding, False for alighting',
    'Line': 'Subway line number (available from 2017 onward, float)',
    'Hourly Columns (05~06 to 24~)': 'Hourly passenger counts for each time interval',
    'Total': 'Total daily passengers per station and boarding type (integer)'
}

for var, description in variables_info.items():
    print(f"• {var:25s}: {description}")

# List hourly columns
hour_cols = [col for col in all_data.columns if '~' in str(col)]
print(f"\nHourly time intervals ({len(hour_cols)} columns):")
print("  " + ", ".join(hour_cols[:10]) + ", ...")



1. KEY VARIABLES AND FIELDS:
----------------------------------------
• Date                     : Timestamp for each daily record (datetime index)
• Station Number           : Unique identifier for subway stations (integer)
• Boarding                 : Boolean flag: True for boarding, False for alighting
• Line                     : Subway line number (available from 2017 onward, float)
• Hourly Columns (05~06 to 24~): Hourly passenger counts for each time interval
• Total                    : Total daily passengers per station and boarding type (integer)

Hourly time intervals (20 columns):
  05~06, 06~07, 07~08, 08~09, 09~10, 10~11, 11~12, 12~13, 13~14, 14~15, ...


In [18]:
# Units and time resolution
print("\n2. UNITS AND TIME RESOLUTION:")
print("-" * 40)

print("• Primary Resolution: Daily records")
print(f"• Date Range: {all_data.index.min().date()} to {all_data.index.max().date()}")
print(f"• Years Covered: {all_data.index.year.nunique()} years ({min(all_data.index.year)}-{max(all_data.index.year)})")
print("• Time Granularity: Hourly intervals within each day")
print("• Units: Passenger counts (integers, no decimals)")
print("• Temporal Scope: Multi-year panel data (2008-2024)")


2. UNITS AND TIME RESOLUTION:
----------------------------------------
• Primary Resolution: Daily records
• Date Range: 2008-01-01 to 2024-12-31
• Years Covered: 17 years (2008-2024)
• Time Granularity: Hourly intervals within each day
• Units: Passenger counts (integers, no decimals)
• Temporal Scope: Multi-year panel data (2008-2024)


In [19]:
# Data volume
print("\n3. DATA VOLUME:")
print("-" * 40)

total_rows = len(all_data)
total_cols = len(all_data.columns)
memory_mb = all_data.memory_usage(deep=True).sum() / (1024 ** 2)

print(f"• Total Rows: {total_rows:,}")
print(f"• Total Columns: {total_cols}")
print(f"• Approximate Memory Size: {memory_mb:.2f} MB")
print(f"• Unique Stations: {all_data['Station Number'].nunique()}")
print(f"• Unique Dates: {all_data.index.nunique()}")
print(f"• Average Records per Day: {total_rows / all_data.index.nunique():.0f}")

# Data volume by period
print("\n  Data Volume Breakdown:")
print(f"  • Pre-2017 data: {sum(len(datasets[y]) for y in range(2008, 2017)):,} rows")
print(f"  • 2017-2024 data: {sum(len(datasets[y]) for y in range(2017, 2025)):,} rows")



3. DATA VOLUME:
----------------------------------------
• Total Rows: 3,378,600
• Total Columns: 24
• Approximate Memory Size: 812.51 MB
• Unique Stations: 564
• Unique Dates: 6210
• Average Records per Day: 544

  Data Volume Breakdown:
  • Pre-2017 data: 1,771,882 rows
  • 2017-2024 data: 1,606,718 rows


In [22]:
# Known limitations
import numpy as np # Import numpy
print("\n4. KNOWN LIMITATIONS:")
print("-" * 40)

limitations = [
    ("Missing Line Information",
     f"Line column contains {all_data['Line'].isnull().sum():,} missing values ({all_data['Line'].isnull().sum()/len(all_data)*100:.1f}%) for pre-2017 data"),

    ("Inconsistent Column Structure",
     "Data structure varies across years (e.g., 24~ column missing in some years, Total column sometimes pre-calculated)"),

    ("Data Aggregation",
     "Data is aggregated at station-hour level; individual trip-level data not available"),

    ("Temporal Gaps",
     "Need to verify completeness of all dates across all stations"),

    ("Encoding and Format Issues",
     "Original data uses EUC-KR encoding; numeric columns sometimes stored as strings with commas"),

    ("Access Constraints",
     "Data sourced from static CSV files; no real-time API or streaming access available"),

    ("Geographic Context Missing",
     "Station names were dropped; only station numbers remain, requiring separate mapping for location analysis")
]

for i, (title, desc) in enumerate(limitations, 1):
    print(f"{i}. {title}:")
    print(f"   {desc}")

# Additional quality metrics
print("\n  Data Quality Metrics:")
numeric_cols = all_data.select_dtypes(include=[np.number]).columns.tolist()
if 'Total' in numeric_cols:
    total_missing = all_data['Total'].isnull().sum()
    print(f"  • Missing Total values: {total_missing:,} ({total_missing/len(all_data)*100:.2f}%)")

# Check for any completely missing columns
completely_missing = [col for col in all_data.columns if all_data[col].isnull().all()]
if completely_missing:
    print(f"  • Completely empty columns: {', '.join(completely_missing)}")


4. KNOWN LIMITATIONS:
----------------------------------------
1. Missing Line Information:
   Line column contains 1,771,882 missing values (52.4%) for pre-2017 data
2. Inconsistent Column Structure:
   Data structure varies across years (e.g., 24~ column missing in some years, Total column sometimes pre-calculated)
3. Data Aggregation:
   Data is aggregated at station-hour level; individual trip-level data not available
4. Temporal Gaps:
   Need to verify completeness of all dates across all stations
5. Encoding and Format Issues:
   Original data uses EUC-KR encoding; numeric columns sometimes stored as strings with commas
6. Access Constraints:
   Data sourced from static CSV files; no real-time API or streaming access available
7. Geographic Context Missing:
   Station names were dropped; only station numbers remain, requiring separate mapping for location analysis

  Data Quality Metrics:
  • Missing Total values: 0 (0.00%)
